**Implementing Feasibility aware Bayesian Optimization using BoTorch**

In [1]:
import torch
import botorch
import pandas as pd
import matplotlib.pyplot as plt

c:\Users\goura\anaconda3\envs\botorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from botorch.models.gp_regression import SingleTaskGP
dtype=torch.double

In [3]:
filename = r'../data/olhs_run1.xlsx'
x_pd = pd.read_excel(filename, sheet_name='Initial Design (OLHS)', header=[0,1], index_col=[0])
y_pd = pd.read_excel(filename, sheet_name='bo_data', header=[0,1], index_col=[0])

#normalizing the inputs
# xmeans = x_pd.mean(axis=0)
# xstddv = x_pd.std(axis=0)
# x_pd_normal = (x_pd - xmeans)/xstddv

# x_pd = x_pd_normal

x_mins = x_pd.min(axis=0).to_numpy()
x_maxs = x_pd.max(axis=0).to_numpy()

x_pd = (x_pd - x_mins)/(x_mins - x_maxs)

#normalize objective value data
y_obj_pd = y_pd.iloc[:, [0,1,2]]
ymeans = y_obj_pd.mean(axis=0)
ystddv = y_obj_pd.std(axis=0)
y_obj_pd_normal = (y_obj_pd - ymeans) / ystddv

y_pd.iloc[:, [0,1,2]] = y_obj_pd_normal

In [11]:
validation_idx = [1,7,15]

train_x_pd = x_pd.drop(validation_idx)
train_y_pd = y_pd.drop(validation_idx)

train_x = torch.tensor(train_x_pd.values,dtype=dtype)
train_y3 = torch.tensor(train_y_pd['Shear Modulus', 'Kpa'].values, dtype=dtype).squeeze()
train_y4 = torch.tensor(train_y_pd['Manufacturability', '--'].values, dtype=torch.long)

In [28]:
train_y4 = torch.reshape(train_y4, (1,-1, 1))
train_y4.shape

torch.Size([1, 17, 1])

In [29]:
from gpytorch.likelihoods import DirichletClassificationLikelihood
dir_likelihood = DirichletClassificationLikelihood(train_y4.squeeze(), learn_additional_noise=False, alpha_epsilon=1e-4)

In [23]:
transformed_y = dir_likelihood.transformed_targets.double()
transformed_y_reshape = torch.reshape(transformed_y, (2, 17, 1))
transformed_y_reshape.shape

torch.Size([2, 17, 1])

In [34]:
train_x = torch.reshape(train_x, (1, 17, 8))
train_x.shape

torch.Size([1, 17, 8])

In [41]:
from botorch.models.transforms import Standardize, Normalize

cls_model = SingleTaskGP(train_X=train_x, train_Y=transformed_y_reshape, likelihood=dir_likelihood, 
                         outcome_transform=Standardize(m=1, batch_shape=transformed_y_reshape.shape[:-2]),
                         input_transform=Normalize(d=8,batch_shape=train_x.shape[:-2]))